In [1]:
import numpy as np 
from PIL import Image 

import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 

import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [3]:
train_data = torchvision.datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
image, label = train_data[0]

In [5]:
image

tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
         [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
         [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
         ...,
         [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
         [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
         [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],

        [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
         [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
         [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
         ...,
         [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
         [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
         [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],

        [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
         [-0.8431, -1.0000, -1.0000,  ..., -0

In [6]:
image.shape

torch.Size([3, 32, 32])

In [7]:
class_names = ["plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [8]:
class NeuralNets(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, 
                               out_channels=12, 
                               kernel_size=5) 
        # input - output  32 - 12 = 27 
        # divided by stride default 1  27/1 = 27 
        # add 1 27 + 1 = 28 
        # output channel (12,28,28)
        self.pool = nn.MaxPool2d(kernel_size=2, 
                                 stride=2)
        #  divide by 2  28 /2 = 14 
        # output (12, 14, 14)
        self.conv2 = nn.Conv2d(12,24,5) # ((14-5)/1)+1 = (24,10,10) -> (24,5,5) -> Flatten -> (24 * 5 * 5)
        self.fc1 = nn.Linear(in_features=24*5*5, 
                             out_features=120)
        self.fc2 = nn.Linear(in_features=120, 
                             out_features=84)
        self.fc3 = nn.Linear(in_features=84,
                             out_features=10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x       

In [9]:
# Setup device-agnostic code 
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [10]:
net = NeuralNets().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
next(net.parameters()).device

device(type='cuda', index=0)

In [11]:
for epoch in range(30):
    print(f"Trianing epoch {epoch}")
    running_loss = 0.0 
    for i, (X,y) in enumerate(train_loader):
        inputs, labels = X.to(device), y.to(device) 
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f'Loss: {running_loss / len(train_loader):.4f}')

Trianing epoch 0
Loss: 2.2996
Trianing epoch 1
Loss: 2.2272
Trianing epoch 2
Loss: 2.0122
Trianing epoch 3
Loss: 1.7966
Trianing epoch 4
Loss: 1.7084
Trianing epoch 5
Loss: 1.6492
Trianing epoch 6
Loss: 1.5965
Trianing epoch 7
Loss: 1.5468
Trianing epoch 8
Loss: 1.5014
Trianing epoch 9
Loss: 1.4610
Trianing epoch 10
Loss: 1.4238
Trianing epoch 11
Loss: 1.3591
Trianing epoch 12
Loss: 1.2156
Trianing epoch 13
Loss: 1.1724
Trianing epoch 14
Loss: 1.1317
Trianing epoch 15
Loss: 1.0926
Trianing epoch 16
Loss: 1.0618
Trianing epoch 17
Loss: 1.0286
Trianing epoch 18
Loss: 1.0027
Trianing epoch 19
Loss: 0.9718
Trianing epoch 20
Loss: 0.9477
Trianing epoch 21
Loss: 0.9214
Trianing epoch 22
Loss: 0.8963
Trianing epoch 23
Loss: 0.8748
Trianing epoch 24
Loss: 0.8512
Trianing epoch 25
Loss: 0.8273
Trianing epoch 26
Loss: 0.8111
Trianing epoch 27
Loss: 0.7892
Trianing epoch 28
Loss: 0.7699
Trianing epoch 29
Loss: 0.7513


In [12]:
torch.save(net.state_dict(), "trained_CIFAR10.path")

In [15]:
net = NeuralNets().to(device)
net.load_state_dict(torch.load("trained_CIFAR10.path"))

<All keys matched successfully>

In [16]:
correct = 0 
total = 0 

net.eval()
with torch.inference_mode():
    for X, y in test_loader:
        images, labels = X.to(device), y.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f"Accuracy: {accuracy}%")

Accuracy: 62.4%


In [22]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_path = ['plane.jpeg', 'horse.jpeg']
images =[load_image(img) for img in image_path]

net.eval() 
with torch.no_grad():
    for image in images:
        image = image.to(device)
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}')

Prediction: plane
Prediction: horse
